In [1]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from collections import Counter

sns.set_style('ticks')

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300
mpl.rc("savefig", dpi=300)

from scipy.special import xlogy

##### Read files and select drugs

In [2]:
# log2_median_ic50, log2_median_ic50_9f, log2_median_ic50_hn, log2_median_ic50_9f_hn, log2_median_ic50_3f_hn, log2_max_conc
ref_type = 'log2_median_ic50_hn' # log2_median_ic50_3f_hn | log2_median_ic50_hn
model_name = 'hn_drug_cw_dw10_100000_model' # hn_drug_cw_dw10_100000_model | hn_drug_cw_dw1_100000_model | hn_drug_cw_dwsim10_100000_model

dosage_shifted = False

In [6]:
norm_type = 'cell_TPM'

current_dir = '../result/HN_model/{}/'.format(norm_type)

In [7]:
drug_info_df = pd.read_csv('../preprocessed_data/GDSC/hn_drug_stat.csv', index_col=0)
drug_info_df.index = drug_info_df.index.astype(str)

drug_id_name_dict = dict(zip(drug_info_df.index, drug_info_df['Drug Name'].values))

drug_info_df.head()

,Drug Name,Synonyms,Target,Target Pathway,Selleckchem Cat#,CAS number,PubCHEM,Others,entropy,max_conc,...,median_ic50_9f,log2_median_ic50_9f,log2_median_ic50_hn,median_ic50_hn,median_ic50_3f_hn,log2_median_ic50_3f_hn,median_ic50_9f_hn,log2_median_ic50_9f_hn,num_sensitive,num_sensitive_hn
Drug ID,,,,,,,,,,,,,,,,,,,,,
1001,AICA Ribonucleotide,"AICAR, N1-(b-D-Ribofuranosyl)-5-aminoimidazole...",AMPK agonist,Metabolism,S1802,2627-69-2,65110,NaN,6.034272,2000.0000,...,206.748380,7.691732,9.939784,982.139588,327.379863,8.354822,109.126621,6.769859,476,27
1003,Camptothecin,"7-Ethyl-10-Hydroxy-Camptothecin, SN-38, Irinot...",TOP1,DNA replication,S1288,7689-03-4,104842,"(SN-38, S4908, 86639-52-3) (Irinotecan, S1198,...",4.609530,0.1000,...,0.002003,-8.963413,-7.587491,0.005199,0.001733,-9.172454,0.000578,-10.757416,688,30
1004,Vinblastine,Velban,Microtubule destabiliser,Mitosis,S1248,143-67-9,6710780,NaN,4.297122,0.1000,...,0.001599,-9.289051,-7.150982,0.007036,0.002345,-8.735945,0.000782,-10.320907,753,33
1006,Cytarabine,"Ara-Cytidine, Arabinosyl Cytosine, U-19920",Antimetabolite,DNA replication,S1648,147-94-4,6253,NaN,6.646594,2.0000,...,0.163032,-2.616771,-1.342632,0.394301,0.131434,-2.927594,0.043811,-4.512557,508,25
1007,Docetaxel,"RP-56976, Taxotere",Microtubule stabiliser,Mitosis,S1148,114977-28-5,148124,NaN,4.220984,0.0125,...,0.000761,-10.358915,-9.792998,0.001127,0.000376,-11.377960,0.000125,-12.962923,584,32


In [8]:
tested_drug_list = [1032, 1007, 133, 201, 1010] + [182, 301, 302] + [1012]
[drug_id_name_dict[str(d)] for d in tested_drug_list]

['Afatinib',
 'Docetaxel',
 'Doxorubicin',
 'Epothilone B',
 'Gefitinib',
 'Obatoclax Mesylate',
 'PHA-793887',
 'PI-103',
 'Vorinostat']

In [9]:
if dosage_shifted:
    single_drug_pred_df = pd.read_csv(current_dir + 'pred_drug_kill_{}_{}_shifted.csv'.format(ref_type, model_name))
else:
    single_drug_pred_df = pd.read_csv(current_dir + 'pred_drug_kill_{}_{}.csv'.format(ref_type, model_name))


single_drug_pred_df.loc[:, 'drug_id'] = single_drug_pred_df.loc[:, 'drug_id'].values.astype(str)
single_drug_pred_df.loc[:, 'drug_name'] = [drug_id_name_dict[d] for d in single_drug_pred_df.loc[:, 'drug_id'].values]

patient_list = sorted(list(set(single_drug_pred_df['patient'])))
# sel_drug_id_list = sorted(list(set(single_drug_pred_df['drug_id'])))

single_drug_pred_df.loc[:, 'cluster_p'] = 1
single_drug_pred_df.loc[:, 'cluster_kill'] = single_drug_pred_df['kill']

single_drug_pred_df.head()

,patient,drug_id,delta,kill,drug_name,cluster_p,cluster_kill
0,HN120,1001,0.941144,34.518773,AICA Ribonucleotide,1,34.518773
1,HN120,1003,1.660548,26.558972,Camptothecin,1,26.558972
2,HN120,1004,1.438976,28.542245,Vinblastine,1,28.542245
3,HN120,1006,1.629793,26.066463,Cytarabine,1,26.066463
4,HN120,1007,3.234392,10.897336,Docetaxel,1,10.897336


##### List all drug pairs

In [10]:
drug_combi_list = []
n_drugs = len(tested_drug_list)

for p in patient_list:
    for x in range(0, n_drugs-1):
        for y in range(x+1, n_drugs):
            drug_x = str(tested_drug_list[x])
            drug_y = str(tested_drug_list[y])

            drug_combi_list += [[p, drug_x, drug_y]]

drug_combi_df = pd.DataFrame(drug_combi_list, columns=['patient', 'A', 'B'])

print (drug_combi_df.shape)
drug_combi_df.head()

(216, 3)


,patient,A,B
0,HN120,1032,1007
1,HN120,1032,133
2,HN120,1032,201
3,HN120,1032,1010
4,HN120,1032,182


##### Get pred and info for each drug

In [11]:
merge_df = pd.merge(drug_combi_df, single_drug_pred_df, how='left', left_on=['patient', 'A'], right_on=['patient', 'drug_id'])
drug_combi_pred_df = pd.merge(merge_df, single_drug_pred_df[['patient', 'drug_id', 'drug_name', 'cluster_kill', 'kill']], how='left', left_on=['patient', 'B'], right_on=['patient', 'drug_id'], suffixes=['_A', '_B'])

In [12]:
drug_combi_pred_df.head()

,patient,A,B,drug_id_A,delta,kill_A,drug_name_A,cluster_p,cluster_kill_A,drug_id_B,drug_name_B,cluster_kill_B,kill_B
0,HN120,1032,1007,1032,3.193168,10.719726,Afatinib,1,10.719726,1007,Docetaxel,10.897336,10.897336
1,HN120,1032,133,1032,3.193168,10.719726,Afatinib,1,10.719726,133,Doxorubicin,59.126943,59.126943
2,HN120,1032,201,1032,3.193168,10.719726,Afatinib,1,10.719726,201,Epothilone B,39.492534,39.492534
3,HN120,1032,1010,1032,3.193168,10.719726,Afatinib,1,10.719726,1010,Gefitinib,16.555392,16.555392
4,HN120,1032,182,1032,3.193168,10.719726,Afatinib,1,10.719726,182,Obatoclax Mesylate,46.369313,46.369313


In [13]:
rows = []
for _, data in drug_combi_pred_df.iterrows():

    
    cluster_kill_A = data['cluster_kill_A']
    cluster_kill_B = data['cluster_kill_B']
    cluster_kill_C = cluster_kill_A + cluster_kill_B - np.multiply(cluster_kill_A/100, cluster_kill_B/100)*100
    kill_C = cluster_kill_C
    
    best_kill = np.max([data['kill_A'], data['kill_B']])
    improve = kill_C - best_kill
    improve_p = (kill_C - best_kill) / best_kill
    
    sum_kill_dif = np.sum(np.abs(cluster_kill_A - cluster_kill_B))
    
    ##### save output #####
    
    rows += [[kill_C, improve, improve_p, sum_kill_dif]]

In [14]:
drug_combi_pred_df = pd.concat([drug_combi_pred_df, pd.DataFrame(rows, columns=['kill_C', 'improve', 'improve_p', 'sum_kill_dif'])], axis=1)
drug_combi_pred_df.head()

,patient,A,B,drug_id_A,delta,kill_A,drug_name_A,cluster_p,cluster_kill_A,drug_id_B,drug_name_B,cluster_kill_B,kill_B,kill_C,improve,improve_p,sum_kill_dif
0,HN120,1032,1007,1032,3.193168,10.719726,Afatinib,1,10.719726,1007,Docetaxel,10.897336,10.897336,20.448897,9.551561,0.876504,0.177611
1,HN120,1032,133,1032,3.193168,10.719726,Afatinib,1,10.719726,133,Doxorubicin,59.126943,59.126943,63.508422,4.381480,0.074103,48.407217
2,HN120,1032,201,1032,3.193168,10.719726,Afatinib,1,10.719726,201,Epothilone B,39.492534,39.492534,45.978769,6.486234,0.164240,28.772809
3,HN120,1032,1010,1032,3.193168,10.719726,Afatinib,1,10.719726,1010,Gefitinib,16.555392,16.555392,25.500425,8.945033,0.540309,5.835667
4,HN120,1032,182,1032,3.193168,10.719726,Afatinib,1,10.719726,182,Obatoclax Mesylate,46.369313,46.369313,52.118375,5.749063,0.123984,35.649587


In [15]:
drug_combi_pred_df = drug_combi_pred_df[['patient', 'drug_id_A', 'drug_name_A', 'drug_id_B', 'drug_name_B', 'cluster_p', 'cluster_kill_A', 'cluster_kill_B', 'kill_A', 'kill_B', 'kill_C', 'improve', 'improve_p', 'sum_kill_dif']]

drug_combi_pred_df.head()

,patient,drug_id_A,drug_name_A,drug_id_B,drug_name_B,cluster_p,cluster_kill_A,cluster_kill_B,kill_A,kill_B,kill_C,improve,improve_p,sum_kill_dif
0,HN120,1032,Afatinib,1007,Docetaxel,1,10.719726,10.897336,10.719726,10.897336,20.448897,9.551561,0.876504,0.177611
1,HN120,1032,Afatinib,133,Doxorubicin,1,10.719726,59.126943,10.719726,59.126943,63.508422,4.381480,0.074103,48.407217
2,HN120,1032,Afatinib,201,Epothilone B,1,10.719726,39.492534,10.719726,39.492534,45.978769,6.486234,0.164240,28.772809
3,HN120,1032,Afatinib,1010,Gefitinib,1,10.719726,16.555392,10.719726,16.555392,25.500425,8.945033,0.540309,5.835667
4,HN120,1032,Afatinib,182,Obatoclax Mesylate,1,10.719726,46.369313,10.719726,46.369313,52.118375,5.749063,0.123984,35.649587


In [16]:
if dosage_shifted:
    drug_combi_pred_df.to_csv(current_dir + 'pred_combi_kill_{}_{}_shifted.csv'.format(ref_type, model_name), index=False)
else:
    drug_combi_pred_df.to_csv(current_dir + 'pred_combi_kill_{}_{}.csv'.format(ref_type, model_name), index=False)